In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv("thedataset/version2.csv")
train
# Get the name of the first column
first_column_name = train.columns[0]

# Select the first column
first_column = train[first_column_name]

# Get the unique values
unique_values = first_column.unique()

# Print the unique values
print(unique_values)

[27 28 29 31 32 30 67 68 70 71]


> This dataset has been provided by 
---
"G. Ediboğlu Bartos, Y. Hoscan, A. Kauer, and É. Hajnal, “A Multilingual Handwritten Character Dataset: T-H-E Dataset,” Acta Polytechnica Hungarica, 2020.(under acceptance)"
> The classes this dataset include ç-27 Ç-66 ğ-28 Ğ-67 ı-29 İ-68 ş-30 Ş-69 ö-31 Ö-70 ü-32 Ü-71 but the classes 66 and 69 which are capital Ç and Ş missing.

> I will be using another dataset to close the gaps and widen the training set.